In [1]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import re, io, nltk, torch 
from nltk.corpus import stopwords
from numpy import linalg as LA
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from sklearn.utils.class_weight import compute_class_weight
from gensim import utils, matutils  
from gensim.corpora.dictionary import Dictionary
from six import string_types, integer_types
from six.moves import zip, range
from numpy import linalg as LA
from sklearn.preprocessing import Normalizer
from scipy import stats
from gensim.utils import deprecated
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray, sum as np_sum, prod, argmax
from collections import Counter
from scipy.spatial import distance
from numpy.random import binomial
from scipy.stats import bernoulli
import numpy as np

import keras
from keras.callbacks import Callback,ModelCheckpoint, ReduceLROnPlateau    
from keras import backend as K
from keras.preprocessing import sequence
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Embedding
from keras.layers import Conv1D, Conv2D, MaxPooling1D, Input
from keras.models import Model
from keras.layers import LSTM,GRU,Dense
from keras.utils import Sequence,to_categorical
from keras.preprocessing.sequence import pad_sequences

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMaskedLM,
    BertForSequenceClassification,
    BertTokenizer,
    CamembertConfig,
    CamembertForMaskedLM,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTConfig,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are',torch.cuda.device_count(),'GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


def my_normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v/norm

#el peso max es igual a la clase mas grande
def det_samples(df, dict_class, c_weights=None):
    conteos= Counter(df)
    aumentos= dict()
    maxi= max(conteos.values())
    for clase in conteos:
        if conteos[clase]==maxi:
            class_maxi=clase
            
    if c_weights is None:        # uniform class weights
        for tipo in conteos:
            actual= conteos[tipo]
            if actual<maxi:
                aumentos[tipo]=int(maxi-actual)
            else:
                aumentos[tipo]=0
        return aumentos   
    else:
        try:
            weigths=my_normalize(c_weights)
            max_wei=max(weigths)
            class_max= list(weigths).index(max_wei)
            str_class=dict_class[class_max]
            samples_max=conteos[str_class]
            
            clase=0
            for peso in weigths:
                k=dict_class[clase]
                if peso == max_wei:
                    aumentos[k]= int(samples_max-conteos[k])
                    #print ("aumentos[k]", aumentos[k], 'sampples min', samples_max, 'conteos[clase]', conteos[clase])
                else:
                    futuro=(samples_max/max_wei)*peso
                    aumentos[k]=int(futuro-conteos[k])
                    #print ("aumentos[k] para pesos no minimos ", aumentos[k], "futuro", futuro, "actual", conteos[k])
                clase+=1
            return aumentos
        except:
            print ("No se ha especificado el vector de pesos par balance de datos")
            
            
stoplist = stopwords.words("english")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def my_cleaner(post):      
    s= re.sub(r"http\S+", "html", post)      
    s= re.sub(r'#\w+ ?', "hashtag", s) 
    s= re.sub(r'@\w+ ?', "user", s)
    s= s.lower()    
    s=emoji_pattern.sub(r'', s)
    sl= list(s.split())  
    sl=[word for word in sl if word not in stoplist]
    s=' '.join([word for word in s.split() if word not in stoplist])
    return s, sl

def Sort_Tuple(tup):  
    return(sorted(tup, key = lambda x: x[1]))   

def predict_data(trained, x_train, x_val, x_test, etiq, etiq_v, etiq_t, name_model):
    etiq = etiq.astype("int")
    trainPredict = trained.predict(x_train, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq, trainPredict)  
    f1=f1_score(etiq, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    print ("")
    print ("Accuracy sobre Train", name_model, ":",acc)  
    print ("F1-score None sobre Train", name_model, ":",f1)
    print ("F1-score macro sobre Train", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t, matriz_t
    

Using TensorFlow backend.
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [2]:
stoplist = stopwords.words("english")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def my_cleaner(post):      
    s= re.sub(r"http\S+", "html", post)   ##########elimina http    
    s= re.sub(r'#\w+ ?', "hashtag", s) ########3
    s= re.sub(r'@\w+ ?', "user", s)##############    
    s= s.lower()    
    s=emoji_pattern.sub(r'', s)
    sl= list(s.split())  
    sl=[word for word in sl if word not in stoplist]
    s=' '.join([word for word in s.split() if word not in stoplist])
    return s, sl

#train =pd.read_csv("../Fine-Tuning/CSV_Stance/train_semeval_raw.csv")
val=pd.read_csv("../Fine-Tuning/CSV_Stance/dev_semeval_raw.csv")
test=pd.read_csv("../Fine-Tuning/CSV_Stance/test_semeval_raw.csv")

print ("Conteo por stance val\n", Counter(val['Label']))
print ("Conteo por stance test\n", Counter(test['Label']))


Conteo por stance val
 Counter({'commenting': 173, 'support': 69, 'questioning': 28, 'denying': 11})
Conteo por stance test
 Counter({'commenting': 778, 'questioning': 106, 'denying': 69, 'support': 68})


In [3]:
MAX_LEN = 50

dir_path='../../model_save/Dos-Fases-all_Stance/'
tokenizer_loaded = BertTokenizer.from_pretrained(dir_path)#'bert-base-uncased')
model_loaded = BertForSequenceClassification.from_pretrained(dir_path, num_labels=4)#'bert-base-uncased',  num_labels=4) 
idx_2_token = tokenizer_loaded.ids_to_tokens
archivo = open(dir_path+'vocab.txt','r')
Word2Index={word.strip():i for i,word in enumerate(archivo.readlines())}
M_BERT_space = model_loaded.bert.embeddings.word_embeddings.weight.detach().cpu().numpy()
transformer = Normalizer().fit(M_BERT_space) 
M_BERT_space=transformer.transform(M_BERT_space)

#sdqc
dict_cl=dict()
dict_cl[0]='support'
dict_cl[1]='denying'
dict_cl[2]='questioning'
dict_cl[3]='commenting'

#sdqc
d_lab=dict()
d_lab["support"]=0
d_lab["denying"]=1
d_lab["questioning"]=2
d_lab["commenting"]=3

etiq_v=[]
for et in val['Label'].values:
    etiq_v.append(d_lab[et])
n_labels_val = np.array(etiq_v)
y_val=to_categorical(n_labels_val,num_classes=4)
sentences_val = val['Tweet'].values
input_ids_val = []
for sent in sentences_val:
    encoded_sent = tokenizer_loaded.encode(my_cleaner(sent)[0],add_special_tokens = False)    
    input_ids_val.append(encoded_sent)
input_ids_val = pad_sequences(input_ids_val, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

etiq_t=[]
for et in test['Label'].values:
    etiq_t.append(d_lab[et])
n_labels_test = np.array(etiq_t)
y_test=to_categorical(n_labels_test,num_classes=4)
sentences_test = test['Tweet'].values
input_ids_test = []
for sent in sentences_test:
    encoded_sent = tokenizer_loaded.encode(my_cleaner(sent)[0],add_special_tokens = False)    
    input_ids_test.append(encoded_sent)
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], shape_val[1], 768))
i=0
for in_id in input_ids_val:
    x_val[i]=M_BERT_space[in_id]
    i+=1
    
shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], shape_test[1], 768))
i=0
for in_id in input_ids_test:
    x_test[i]=M_BERT_space[in_id]
    i+=1

x_test.shape

(1021, 50, 768)

In [7]:
#SMOTE
shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], 768))
i=0
for in_id in input_ids_val:
    vector=np.zeros(768)
    n = np.sum(in_id != 0)
    for idv in in_id:
        if idv!=0:
            vector+=M_BERT_space[idv]
    x_val[i]=(vector/n)
    i+=1  

shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], 768))
i=0
for in_id in input_ids_test:
    vector=np.zeros(768)
    n = np.sum(in_id != 0)
    for idv in in_id:
        if idv!=0:
            vector+=M_BERT_space[idv]
    x_test[i]=(vector/n)
    i+=1  

In [13]:
from imblearn.over_sampling import SMOTE

x_val=np.expand_dims(x_val, axis=-1)
x_test=np.expand_dims(x_test, axis=-1)

In [14]:
x_test.shape

(1021, 768, 1)

In [9]:
def focal_loss(gamma=2., weights=1):   #weights np.asarray()
    weights= K.variable(weights)
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(),1)
        y_pred = K.clip(y_pred,K.epsilon(),1)
        return - K.sum(weights* K.pow(1. - y_pred, gamma)* y_true * K.log(y_pred), axis=-1) 
    return focal_loss_fixed

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []

    def on_epoch_end(self, batch, logs={}):
        prob = self.model.predict(self.validation_data[0])
        predict = np.squeeze(prob>=0.5)*1
        targ = np.squeeze(self.validation_data[1])
        f1s = f1_score(targ, predict, average='macro')
        self.val_f1s.append(f1s)
        #print(" - val_f1: %f " %(f1s))
        return

def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):
    
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
def predecir_modelos(bs,predichos_all):  #best fit
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        indice=final[i]%4
        predichos_new.append(int(indice))
    return predichos_new,final,confianzas    

def predecir_modelos_norm(bs,predichos_all):  #normalizado
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:   
        aux=np.ones(4)
        for i in range(1,6): #for machine pred
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas
    
def predecir_modelos_average(bs,predichos_all): #average
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:
        aux=np.zeros(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/5.0 #dividido en el total de machines
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas


def predict_data(trained, x_val, x_test, etiq_v, etiq_t, name_model):
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t #, matriz_t

In [10]:
x_test.shape

(1021, 768)

# Baselines masking=0.0 Sin class weights

In [11]:
prob=0.0
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/0.0_wo/cnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5
#exec_baselines/0.0/cnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5

for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'_wo/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'_wo/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'_wo/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'_wo/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'_wo/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.594306049822064
F1-macro: 0.4042304566125481
F1-score SDQC: 0.4042304566125481

Committee Norm
Accuracy: 0.7117437722419929
F1-macro: 0.49047701604913807
F1-score SDQC: 0.49047701604913807

Committee Voting
Accuracy: 0.693950177935943
F1-macro: 0.4650624965585595
F1-score SDQC: 0.4650624965585595
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.584720861900098
F1-macro: 0.29881699037174525
F1-score SDQC: 0.29881699037174525

Committee Norm
Accuracy: 0.7590597453476984
F1-macro: 0.389546561168594
F1-score SDQC: 0.389546561168594

Committee Voting
Accuracy: 0.7463271302644466
F1-macro: 0.37464293061048926
F1-score SDQC: 0.37464293061048926

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit

In [12]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.6441281138790035
[VAL] F1-score SDQC Best comité: [0.49627097 0.01904762 0.51844004 0.742866  ]
[VAL] F1 macro Best comité: 0.44415615752984833
-----------
[VAL] Acc Norm comité: 0.7010676156583631
[VAL] F1-score SDQC Norm comité: [0.49230097 0.         0.609494   0.79039023]
[VAL] F1 macro Norm comité: 0.4730462989917993
-----------
[VAL] Acc Voting comité: 0.7053380782918149
[VAL] F1-score SDQC Voting comité: [0.52059678 0.         0.6046009  0.79308641]
[VAL] F1 macro Voting comité: 0.479571022779924

[TEST] Acc Best comité: 0.6718903036238981
[TEST] F1-score SDQC Best comité: [0.09774264 0.02933096 0.47546846 0.7987347 ]
[TEST] F1 macro Best comité: 0.3503191902649803
-----------
[TEST] Acc Norm comité: 0.7490695396669931
[TEST] F1-score SDQC Norm comité: [0.09614455 0.0056338  0.49182102 0.85250368]
[TEST] F1 macro Norm comité: 0.36152576231857725
-----------
[TEST] Acc Voting comité: 0.7428011753183155
[TEST] F1-score SDQC Voting comité: [0.09052637 0.   

In [22]:
np.sum(fs_macro_test['cnn1'])/5

0.27926443846452165

### No data augmentation with class weights

In [23]:
prob=0.0
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.5765124555160143
F1-macro: 0.4262522159817738
F1-score SDQC: 0.4262522159817738

Committee Norm
Accuracy: 0.6476868327402135
F1-macro: 0.5029932746171377
F1-score SDQC: 0.5029932746171377

Committee Voting
Accuracy: 0.6263345195729537
F1-macro: 0.4921468203386198
F1-score SDQC: 0.4921468203386198
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.5935357492654261
F1-macro: 0.35860894088445894
F1-score SDQC: 0.35860894088445894

Committee Norm
Accuracy: 0.6650342801175319
F1-macro: 0.41753876314778104
F1-score SDQC: 0.41753876314778104

Committee Voting
Accuracy: 0.6258570029382958
F1-macro: 0.4052692492320878
F1-score SDQC: 0.4052692492320878

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best 

In [24]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.5572953736654804
[VAL] F1-score SDQC Best comité: [0.44928978 0.06577541 0.49594198 0.6686333 ]
[VAL] F1 macro Best comité: 0.4199101187824566
-----------
[VAL] Acc Norm comité: 0.6377224199288256
[VAL] F1-score SDQC Norm comité: [0.46990669 0.10805169 0.63969022 0.73832574]
[VAL] F1 macro Norm comité: 0.4889935831764824
-----------
[VAL] Acc Voting comité: 0.6241992882562277
[VAL] F1-score SDQC Voting comité: [0.49810547 0.1113421  0.64751733 0.71791995]
[VAL] F1 macro Voting comité: 0.49372121339014974

[TEST] Acc Best comité: 0.568462291870715
[TEST] F1-score SDQC Best comité: [0.09960681 0.12794124 0.42696566 0.71447241]
[TEST] F1 macro Best comité: 0.3422465314176061
-----------
[TEST] Acc Norm comité: 0.6703232125367287
[TEST] F1-score SDQC Norm comité: [0.09995262 0.15183126 0.55804336 0.79726147]
[TEST] F1 macro Norm comité: 0.4017721784762949
-----------
[TEST] Acc Voting comité: 0.6366307541625857
[TEST] F1-score SDQC Voting comité: [0.11186773 0.1496

### Augmentation 0.15

In [25]:
prob=0.15
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.5444839857651246
F1-macro: 0.46378662596849235
F1-score SDQC: 0.46378662596849235

Committee Norm
Accuracy: 0.7046263345195729
F1-macro: 0.544000074000074
F1-score SDQC: 0.544000074000074

Committee Voting
Accuracy: 0.701067615658363
F1-macro: 0.5442445286195285
F1-score SDQC: 0.5442445286195285
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.5817825661116552
F1-macro: 0.3802789539438112
F1-score SDQC: 0.3802789539438112

Committee Norm
Accuracy: 0.7277179236043095
F1-macro: 0.4069071187843518
F1-score SDQC: 0.4069071187843518

Committee Voting
Accuracy: 0.7306562193927522
F1-macro: 0.40883005991758814
F1-score SDQC: 0.40883005991758814

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit

In [26]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.5416370106761565
[VAL] F1-score SDQC Best comité: [0.5041248  0.1100757  0.5501704  0.60949355]
[VAL] F1 macro Best comité: 0.4434661124481993
-----------
[VAL] Acc Norm comité: 0.6505338078291816
[VAL] F1-score SDQC Norm comité: [0.53546343 0.13219002 0.61149466 0.73660707]
[VAL] F1 macro Norm comité: 0.5039387934301434
-----------
[VAL] Acc Voting comité: 0.6754448398576512
[VAL] F1-score SDQC Voting comité: [0.53017985 0.14066487 0.62281481 0.76788696]
[VAL] F1 macro Voting comité: 0.515386624916684

[TEST] Acc Best comité: 0.540058765915769
[TEST] F1-score SDQC Best comité: [0.08281138 0.14256432 0.46463804 0.66971359]
[TEST] F1 macro Best comité: 0.3399318332986885
-----------
[TEST] Acc Norm comité: 0.6763956904995103
[TEST] F1-score SDQC Norm comité: [0.07989772 0.1446916  0.51949631 0.7968803 ]
[TEST] F1 macro Norm comité: 0.38524148310491974
-----------
[TEST] Acc Voting comité: 0.7163565132223311
[TEST] F1-score SDQC Voting comité: [0.07965093 0.14311

### Augmentation 0.5

In [27]:
prob=0.5
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.6120996441281139
F1-macro: 0.48186221303056387
F1-score SDQC: 0.48186221303056387

Committee Norm
Accuracy: 0.6583629893238434
F1-macro: 0.5323841495611954
F1-score SDQC: 0.5323841495611954

Committee Voting
Accuracy: 0.6476868327402135
F1-macro: 0.5339278480591296
F1-score SDQC: 0.5339278480591296
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.5984329089128305
F1-macro: 0.28561820825705553
F1-score SDQC: 0.28561820825705553

Committee Norm
Accuracy: 0.66307541625857
F1-macro: 0.32721590304482195
F1-score SDQC: 0.32721590304482195

Committee Voting
Accuracy: 0.6807051909892262
F1-macro: 0.3672172773085848
F1-score SDQC: 0.3672172773085848

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best 

In [28]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.5978647686832739
[VAL] F1-score SDQC Best comité: [0.55932935 0.11930057 0.49059695 0.69569185]
[VAL] F1 macro Best comité: 0.46622967968235196
-----------
[VAL] Acc Norm comité: 0.6583629893238433
[VAL] F1-score SDQC Norm comité: [0.58930041 0.15820794 0.53416229 0.74450148]
[VAL] F1 macro Norm comité: 0.5065430308013066
-----------
[VAL] Acc Voting comité: 0.6676156583629893
[VAL] F1-score SDQC Voting comité: [0.57647713 0.15594133 0.52715549 0.75800546]
[VAL] F1 macro Voting comité: 0.5043948514723415

[TEST] Acc Best comité: 0.6266405484818806
[TEST] F1-score SDQC Best comité: [0.06349088 0.1453796  0.33412465 0.77529   ]
[TEST] F1 macro Best comité: 0.3295712822341257
-----------
[TEST] Acc Norm comité: 0.6999020568070519
[TEST] F1-score SDQC Norm comité: [0.05277576 0.12650848 0.38033802 0.82358979]
[TEST] F1 macro Norm comité: 0.34580301217351916
-----------
[TEST] Acc Voting comité: 0.7185112634671891
[TEST] F1-score SDQC Voting comité: [0.04775996 0.12

### Augmentation 0.85

In [29]:
prob=0.85
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.4875444839857651
F1-macro: 0.44524332875400685
F1-score SDQC: 0.44524332875400685

Committee Norm
Accuracy: 0.5765124555160143
F1-macro: 0.4773203494265976
F1-score SDQC: 0.4773203494265976

Committee Voting
Accuracy: 0.6334519572953736
F1-macro: 0.5013640009942604
F1-score SDQC: 0.5013640009942604
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.48579823702252695
F1-macro: 0.3160200008115438
F1-score SDQC: 0.3160200008115438

Committee Norm
Accuracy: 0.6366307541625857
F1-macro: 0.3691808669052116
F1-score SDQC: 0.3691808669052116

Committee Voting
Accuracy: 0.7012732615083251
F1-macro: 0.3888591608984444
F1-score SDQC: 0.3888591608984444

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best F

In [30]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.5651245551601424
[VAL] F1-score SDQC Best comité: [0.52836536 0.09871309 0.47961841 0.65325739]
[VAL] F1 macro Best comité: 0.43998856175124923
-----------
[VAL] Acc Norm comité: 0.6377224199288256
[VAL] F1-score SDQC Norm comité: [0.57127392 0.11585295 0.54043676 0.72889176]
[VAL] F1 macro Norm comité: 0.489113846661442
-----------
[VAL] Acc Voting comité: 0.6334519572953736
[VAL] F1-score SDQC Voting comité: [0.5628745  0.12057731 0.52487986 0.72457448]
[VAL] F1 macro Voting comité: 0.48322653724754544

[TEST] Acc Best comité: 0.6084231145935357
[TEST] F1-score SDQC Best comité: [0.07976099 0.10426943 0.40958427 0.75334007]
[TEST] F1 macro Best comité: 0.3367386911847197
-----------
[TEST] Acc Norm comité: 0.6897159647404505
[TEST] F1-score SDQC Norm comité: [0.06928257 0.12182337 0.47238586 0.81424051]
[TEST] F1 macro Norm comité: 0.36943307795389824
-----------
[TEST] Acc Voting comité: 0.7008814887365328
[TEST] F1-score SDQC Voting comité: [0.06545552 0.13

### SMOTE

In [15]:
path='../Operador Data Augmentation/exec_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
#cnn1_4239-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+'SMOTE/cnn1_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+'SMOTE/cnn2_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+'SMOTE/rnn1_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+'SMOTE/rnn2_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+'SMOTE/rnn3_'+str(j+1)+'-exec_w4_app1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.6120996441281139
F1-macro: 0.5002924631620869
F1-score SDQC: 0.5002924631620869

Committee Norm
Accuracy: 0.6512455516014235
F1-macro: 0.5238395854866276
F1-score SDQC: 0.5238395854866276

Committee Voting
Accuracy: 0.6370106761565836
F1-macro: 0.510019658310196
F1-score SDQC: 0.510019658310196
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.6219392752203722
F1-macro: 0.3745742433017494
F1-score SDQC: 0.3745742433017494

Committee Norm
Accuracy: 0.653281096963761
F1-macro: 0.39839050503507156
F1-score SDQC: 0.39839050503507156

Committee Voting
Accuracy: 0.6571988246816847
F1-macro: 0.4007919849011342
F1-score SDQC: 0.4007919849011342

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
A

In [16]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.6405693950177936
[VAL] F1-score SDQC Best comité: [0.50278557 0.30778459 0.5810471  0.72815624]
[VAL] F1 macro Best comité: 0.5299433725859554
-----------
[VAL] Acc Norm comité: 0.6697508896797153
[VAL] F1-score SDQC Norm comité: [0.51621945 0.29572193 0.62066656 0.75723694]
[VAL] F1 macro Norm comité: 0.5474612169273338
-----------
[VAL] Acc Voting comité: 0.6654804270462633
[VAL] F1-score SDQC Voting comité: [0.50775309 0.29831933 0.61296126 0.75344016]
[VAL] F1 macro Voting comité: 0.5431184600270799

[TEST] Acc Best comité: 0.662095984329089
[TEST] F1-score SDQC Best comité: [0.23028205 0.09138268 0.44885645 0.79097373]
[TEST] F1 macro Best comité: 0.3903737254845278
-----------
[TEST] Acc Norm comité: 0.6926542605288932
[TEST] F1-score SDQC Norm comité: [0.24139254 0.08032411 0.46617391 0.812957  ]
[TEST] F1 macro Norm comité: 0.4002118910115049
-----------
[TEST] Acc Voting comité: 0.6908912830558277
[TEST] F1-score SDQC Voting comité: [0.23708623 0.07609

In [17]:
np.sum(fs_macro_test['cnn1'])/5

0.38840496421734094